In [ ]:
!pip install pycuda

In [2]:
import pycuda.compiler as comp
import pycuda.driver as cuda
import pycuda.autoinit
import numpy
import time
import math

class Net():
    def __init__(self):
        super(Net, self).__init__()

    def setSize(self, layers):
      self.layers = layers
      self.numberOfNodes = 0
      for i in range(len(self.layers)):
        self.numberOfNodes += self.layers[i]

      self.nodesGrad = numpy.zeros((self.numberOfNodes, 1),dtype=numpy.float32)
      self.nodes = numpy.zeros((self.numberOfNodes, 1),dtype=numpy.float32)
      self.loss = numpy.zeros((self.numberOfNodes, 1),dtype=numpy.float32)

      self.numberOfWeights = 0
      for i in range(len(self.layers)-1):
        self.numberOfWeights += self.layers[i] * self.layers[i+1]

      self.grads = numpy.zeros((self.numberOfWeights, 1),dtype=numpy.float32)
      self.weights = numpy.zeros((self.numberOfWeights, 1),dtype=numpy.float32)
      self.weightsLoss = numpy.zeros((self.numberOfWeights, 1),dtype=numpy.float32)

    def copyToDevice(self):
      self.weights_gpu = cuda.mem_alloc(self.weights.nbytes)
      self.nodes_gpu = cuda.mem_alloc(self.nodes.nbytes)
      self.grads_gpu = cuda.mem_alloc(self.grads.nbytes)
      self.nodesGrad_gpu = cuda.mem_alloc(self.nodesGrad.nbytes)
      self.loss_gpu = cuda.mem_alloc(self.loss.nbytes)

      self.weightsLoss_gpu = cuda.mem_alloc(self.weightsLoss.nbytes)

      cuda.memcpy_htod(self.weights_gpu,self.weights)
      cuda.memcpy_htod(self.nodes_gpu,self.nodes)
      cuda.memcpy_htod(self.grads_gpu,self.grads)
      cuda.memcpy_htod(self.nodesGrad_gpu,self.nodesGrad)
      cuda.memcpy_htod(self.loss_gpu,self.loss)

      cuda.memcpy_htod(self.weightsLoss_gpu,self.weightsLoss)

    def initWeights(self):
      start = 0
      for x in range(len(self.layers)-1):
        numberOfWeights = self.layers[x] * self.layers[x+1]
        layerSize = self.layers[x]
        for y in range(numberOfWeights):
          self.weights[start + y] = numpy.random.uniform() * (2 / numpy.sqrt(layerSize)) - 1 / numpy.sqrt(layerSize)
        start += numberOfWeights

    def free(self):
      self.weights_gpu.free()
      self.nodes_gpu.free()
      self.grads_gpu.free()
      self.nodesGrad_gpu.free()
      self.loss_gpu.free()
      self.weightsLoss_gpu.free()

    def optimize(self):
      length = len(self.weights)
      bx,by,gx,gy = self.getBlockAndGridSize(length,1)
      optimize(self.weights_gpu, self.grads_gpu,self.learningRate, numpy.int32(length), block=(bx,by,1),grid=(gx,gy))

    def zero_grad(self):
      length = len(self.weights)
      bx,by,gx,gy = self.getBlockAndGridSize(length,1)
      reset_values(self.grads_gpu,numpy.int32(length),block=(bx,by,1),grid=(gx,gy))
  
    def getLoss(self, answer):
        numberOfLayers = len(self.layers)
        start = numpy.int32(self.numberOfNodes - self.layers[numberOfLayers-1])
        check_answer(training_correct_gpu, self.nodes_gpu, start, numpy.int32(answer),block=(1,1,1))

        lengthx = self.layers[numberOfLayers-1]
        lengthy = 1

        bx,by,gx,gy = self.getBlockAndGridSize(lengthx,lengthy)
        get_output_loss(self.loss_gpu, self.nodes_gpu, start, numpy.int32(answer),
                        block=(bx,by,1),grid=(gx,gy))

    def backward(self):
      length = len(self.nodesGrad)

      bx,by,gx,gy = self.getBlockAndGridSize(length,1)

      der_sigmoid(self.nodesGrad_gpu,self.nodes_gpu, numpy.int32(length),block=(bx,by,1),grid=(gx,gy))
      numberOfLayers = len(self.layers)

      startw0 = numpy.int32(len(self.weights) - (self.layers[numberOfLayers-1] * self.layers[numberOfLayers-2]))
      startn1 = numpy.int32(self.numberOfNodes - self.layers[numberOfLayers-1])
      startn0 = startn1 - numpy.int32(self.layers[numberOfLayers-2])
      lengthn0 = self.layers[numberOfLayers-2]
      lengthn1 = self.layers[numberOfLayers-1]
      lengthw1 = self.layers[numberOfLayers-2] * self.layers[numberOfLayers-1]
      
      lengthx = lengthn0
      lengthy = lengthn1

      bx,by,gx,gy = self.getBlockAndGridSize(lengthx,lengthy)

      startC = startn0
      startD = startw0
      startA = startn1
      startB = startA
      ncB = numpy.int32(lengthn0)
      nrA = numpy.int32(lengthn1)
      multiply_them_index_add(self.grads_gpu, self.loss_gpu, self.nodesGrad_gpu,
       self.nodes_gpu, startA, startB, startC, startD, ncB, nrA,
        block=(bx,by,1), grid=(gx,gy)) 
      

      startw1 = numpy.int32(len(self.weights))
      for y in range(len(self.layers)-2):

        startw1 -= numpy.int32(self.layers[numberOfLayers-1-y] * self.layers[numberOfLayers-2-y])
        startw0 -= numpy.int32(self.layers[numberOfLayers-2-y] * self.layers[numberOfLayers-3-y])

        startn1 -= numpy.int32(self.layers[numberOfLayers-2-y])
        startn0 -= numpy.int32(self.layers[numberOfLayers-3-y])

        lengthn0 = self.layers[numberOfLayers-3-y]
        lengthn1 = self.layers[numberOfLayers-2-y]
        lengthn2 = self.layers[numberOfLayers-1-y]

        lengthw1 = self.layers[numberOfLayers-3-y] * self.layers[numberOfLayers-2-y]

        length = lengthw1
        bx,by,gx,gy = self.getBlockAndGridSize(length,1)
        startD = startn1
        startA = startw1
        startB = startw1
        array_mulitply(self.weightsLoss_gpu,self.weights_gpu,self.grads_gpu,startD,startA,startB,numpy.int32(length)
        ,block=(bx,1,1),grid=(gx,1))

        startA = startn1
        length = lengthn1
        startD = startn0
        bx,by,gx,gy = self.getBlockAndGridSize(length,1)
        numberOfNodesInLayer = numpy.int32(lengthn2)
        get_node_loss(self.loss_gpu,self.weightsLoss_gpu,numberOfNodesInLayer,startA,startD,
                      numpy.int32(length),block=(bx,1,1),grid=(gx,1))

        startA = startn0
        startB = startn1
        startC = startn0
        startD = startw0
        lengthx = lengthn0
        lengthy = lengthn1

        bx,by,gx,gy = self.getBlockAndGridSize(lengthx,lengthy)

        multiply_them_index_add(self.grads_gpu,self.loss_gpu,self.nodesGrad_gpu, self.nodes_gpu,startA,startB,startC,startD,numpy.int32(lengthx),numpy.int32(lengthy),
                  block=(bx,by,1),grid=(gx,gy))        

    def forward(self, input):
      length = self.layers[0]

      bx,by,gx,gy = self.getBlockAndGridSize(length,1)
  
      copy(self.nodes_gpu, input, numpy.int32(0), numpy.int32(0), numpy.int32(length), block=(bx,by,1), grid=(gx,gy))

      startn0 = numpy.int32(0)
      startn1 = numpy.int32(self.layers[0])
      startw = numpy.int32(0)

      for x in range(len(self.layers)-1):
        
        if x > 0:
          startw += numpy.int32(self.layers[x-1] * self.layers[x])
          startn1 += numpy.int32(self.layers[x])
          startn0 += numpy.int32(self.layers[x-1])


        n = self.layers[x]
        n_NP = numpy.int32(n)
        nrA = numpy.int32(self.layers[x+1])

        bx,by,gx,gy = self.getBlockAndGridSize(1,self.layers[x+1])

        multiply_them_index(self.nodes_gpu, self.weights_gpu, self.nodes_gpu, n_NP, numpy.int32(bx) 
        ,nrA , startn0, startn1,
                              startw, block=(bx,by,1), grid=(gx,gy))

        length = self.layers[x+1]
        bx,by,gx,gy = self.getBlockAndGridSize(length,1)

        sigmoid(self.nodes_gpu,startn1,numpy.int32(length),
                      block=(bx,by,1), grid=(gx,gy))
      return

    def getBlockAndGridSize(self,lengthx,lengthy):
      bx = lengthx
      by = lengthy
      gx = 1
      gy = 1
      if bx > MAX_THREADS_PER_BLOCK:
        gx = math.ceil(bx / MAX_THREADS_PER_BLOCK)
        bx = MAX_THREADS_PER_BLOCK

      if by > MAX_THREADS_PER_BLOCK:
        gy = math.ceil(by / MAX_THREADS_PER_BLOCK)
        by = MAX_THREADS_PER_BLOCK

      if bx * by > MAX_THREADS_PER_BLOCK:
        if by > bx:
          bx = math.floor(MAX_THREADS_PER_BLOCK / by)
          gx = math.ceil(lengthx / bx)
        else:
          by = math.floor(MAX_THREADS_PER_BLOCK / bx)
          gy = math.ceil(lengthy / by) 
      return bx,by,gx,gy

mod = comp.SourceModule(
    """
  __global__ void multiply_them_index(float *nodesD, float *weights, float *nodesA, int ncA, int ncB, int nrA, int startn0, int startD, int startW)
{
  int row = threadIdx.y + blockDim.y * blockIdx.y;
  int col = threadIdx.x + blockDim.x * blockIdx.x;
  float t = 0;
  if(col < ncB && row < nrA)
  {
  for(int i = 0; i < ncA; i++){
    t += weights[startW + (row * ncA) + i] * nodesA[startn0 + col + (i * ncB)];
  }
    nodesD[startD + (row * ncB) + col] = t;
  }
}


__global__ void multiply_them_index_add(float *d, float *a, float *b ,float *c, int startA, int startB, int startC, int startD, int ncB, int nrA)
{
  int row = threadIdx.y + blockDim.y * blockIdx.y;
  int col = threadIdx.x + blockDim.x * blockIdx.x;
  if(col < ncB && row < nrA)
  {
    d[startD + (row * ncB) + col] += a[startA + row] * b[startB + row] * c[startC + col];
  }
}

__global__ void optimize(float *d, float *a, float lr, int length)
{
  int i = threadIdx.x + blockIdx.x * blockDim.x;
  if(i < length)
  {
  d[i] = (lr * -a[i]) + d[i];
  }
}


__global__ void array_mulitply(float *d, float *a, float *b, int startD, int startA, int startB, int length)
{
  const int i = threadIdx.x + (blockDim.x * blockIdx.x);
  if(i < length)
  {
  d[startD + i] = a[startA + i] * b[startB + i];
  }
}


__global__ void get_output_loss(float *d, float *o, int start, int a)
{
  int i = threadIdx.x;
  if(i == a) {
    d[start + i] = o[start + i] - 1;
  } else {
    d[start + i] = o[start + i];
  }
}

__global__ void get_node_loss(float *d, float *a, int n, int startA, int startD, int length)
{
  int i = threadIdx.x + blockDim.x * blockIdx.x;
  float t = 0;
  for(int j = 0; j < n; j++) 
  {
    if(i < length)
    {
    t += a[startA + i + j*length];
    }
  }
  if(i < length)
  { 
  d[startD + i] = t / n;
  }
}

__global__ void reset_values(float *d, int length)
{
  int i = threadIdx.x + blockDim.x * blockIdx.x;
  if(i < length)
  {
    d[i] = 0;
  }
}


__global__ void check_answer(int *a, float *output, int start,int answer)
{
  for(int i = 0; i < 10; i++)
  {
    if(i != answer && output[start + i] >= output[start + answer])
    {
      return;
    }
  }
  a[0] = a[0] + 1;
}

__global__ void sigmoid(float *d, int start, int length)
{
  const int i = threadIdx.x + blockDim.x * blockIdx.x;
  if(i < length)
  {
    d[start + i] = 1 / (1 + exp(-d[start + i]));
  }
}

__global__ void der_sigmoid(float *d, float *a, int length)
{
  const int i = threadIdx.x + blockIdx.x * blockDim.x;
  if(i < length)
  {
    d[i] = a[i] * (1 - a[i]);
  }
}

__global__ void copy(float *d, float *a, int startA, int startD, int length)
{
  int i = threadIdx.x + blockIdx.x * blockDim.x;
  if(i < length)
  {
    d[i + startD] = a[i + startA];
  }
}
"""
)

MAX_THREADS_PER_BLOCK = \
    cuda.Device(0).get_attribute(pycuda._driver.device_attribute.MAX_THREADS_PER_BLOCK)

multiply_them_index = mod.get_function("multiply_them_index")
multiply_them_index_add = mod.get_function("multiply_them_index_add")
optimize = mod.get_function("optimize")
sigmoid = mod.get_function("sigmoid")
der_sigmoid = mod.get_function("der_sigmoid")
array_mulitply = mod.get_function("array_mulitply")
get_output_loss = mod.get_function("get_output_loss")
get_node_loss = mod.get_function("get_node_loss")
reset_values = mod.get_function("reset_values")
check_answer = mod.get_function("check_answer")
copy = mod.get_function("copy")


training_correct = numpy.zeros((1),dtype=numpy.int32)
training_correct_gpu = cuda.mem_alloc(training_correct.nbytes)
cuda.memcpy_htod(training_correct_gpu,training_correct)

test_correct = numpy.zeros((1),dtype=numpy.int32)
test_correct_gpu = cuda.mem_alloc(test_correct.nbytes)
cuda.memcpy_htod(test_correct_gpu,test_correct)

In [ ]:
from torchvision import datasets, transforms

transform=transforms.Compose([
transforms.ToTensor()
])

trainDataset = datasets.MNIST('../data', train=True, download=True,
                transform=transform)
testDataset = datasets.MNIST('../data', train=False,
                transform=transform)

img_train = trainDataset.data.numpy()
img_train = img_train / 255
img_train = numpy.float32(img_train)

img_test = testDataset.data.numpy()
img_test = img_test / 255
img_test = numpy.float32(img_test)

label_train2 = trainDataset.targets

trainImg32 = img_train[0].astype(numpy.float32)
img_gpu = cuda.mem_alloc(trainImg32.nbytes)

cudaNet = Net()

cudaNet.setSize([784,256,10])
cudaNet.initWeights()
cudaNet.learningRate = numpy.float32(1.0)
cudaNet.copyToDevice()

batchSize = 1
for epoch in range(1):
  print("\nEPOCH",epoch,"\n")
  start_time = time.time()
  for i in range(len(img_train)): 
    trainImg32 = img_train[i].astype(numpy.float32)
    cuda.memcpy_htod(img_gpu,trainImg32)

    cudaNet.forward(img_gpu)
    cudaNet.getLoss(label_train2[i])
    cudaNet.backward()
    
    if i % batchSize == 0 or i == (len(img_train) - 1):
      cudaNet.optimize()      
      cudaNet.zero_grad()  

  print("--- %s seconds ---" % (time.time() - start_time))
  cuda.memcpy_dtoh(training_correct,training_correct_gpu)
  reset_values(training_correct_gpu,numpy.int32(1),block=(1,1,1))
  print("train dataset: correct = ",(training_correct[0]/len(img_train)))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PytorchNet(nn.Module):
    def __init__(self):
        super(PytorchNet, self).__init__()
        self.fc1 = nn.Linear(784, 256, False)
        self.fc2 = nn.Linear(256, 10, False)


    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        return F.sigmoid(x)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            targetOutput = torch.tensor([0,0,0,0,0,0,0,0,0,0])
            targetOutput[target[0].item()] = 1
            targetOutput = targetOutput.to(device)
            output = model(data)
            test_loss += F.smooth_l1_loss(output, targetOutput)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
torch.manual_seed(1)
device = torch.device("cuda")

transform=transforms.Compose([
transforms.ToTensor()
])

test_loader = torch.utils.data.DataLoader(testDataset)

model = PytorchNet().to(device)

# copy weights from cudaNet
cuda.memcpy_dtoh(cudaNet.weights,cudaNet.weights_gpu)
start = 0
i = 0
for fc in model.children():
  with torch.no_grad():
    for y in range(cudaNet.layers[i]):
      for x in range(cudaNet.layers[i+1]):
        fc.weight[x, y] = torch.cuda.FloatTensor(cudaNet.weights[start + y + x*cudaNet.layers[i]])
    start += cudaNet.layers[i] * cudaNet.layers[i+1]
    i += 1

cudaNet.free()
test(model, device, test_loader)
